In [51]:
import pandas as pd
import requests
import re

In [52]:
pd.set_option('display.max_columns', None)

Ejercicios de API's

Enunciado - Obtener datos energéticos usando la API de Red Eléctrica.

Descripción: La empresa para la que trabajamos dispone de una base de datos que contiene información sobre el número de personas censadas en las distintas Comunidades Autónomas españolas desde 1998 hasta 2022. El objetivo es enriquecer estos datos con información sobre la generación de energía renovable y no renovable, utilizando la API de REData, la cual proporciona información sobre el mercado energético en España. Puedes encontrar la documentación de la API 

# 1. Extracción de Datos a Nivel Nacional:

Extraer datos de generación de energía renovable y no renovable a nivel nacional desde el 1 de enero de 2011 hasta el 31 de diciembre de 2022. La información debe ser recopilada a nivel mensual. -------SOLO ME DEJA MAX 24 MESES POR REQUEST Lo he cambiado a 2021-2022
--------Detalles del error: {"errors":[{"code":413,"status":"400","title":"Widget bad request","detail":"Petici\u00f3n fuera de rango. L\u00edmite a nivel mensual: 24."}]}---------

Transformar los datos para incluir una columna nueva que identifique si la energía es renovable o no renovable.

Almacenar los datos en un DataFrame con la siguiente estructura: 

In [53]:
def llamar_api(url, headers, params = None):
    # Realizar la solicitud GET pasando la URL y los encabezados.
    llamada = requests.get(url, headers=headers,params=params)
    # Imprimir el código de estado de la respuesta para verificar si fue exitosa.
    print(f"La llamada a la API nos ha dado una respuesta de tipo: {llamada.status_code}")
    # Si el código de estado es diferente de 200, imprimir la razón del fallo.
    if llamada.status_code != 200:
        print(f"El motivo por el que la llamada falló es {llamada.reason}")
        print("Detalles del error:", llamada.text)   #llamada.text me da detalles más especificos del error y así poder arreglarlo.
    else:
        # Si la llamada fue exitosa, devolver la respuesta en formato JSON.
        return llamada.json()   

# Definir el endpoint específico que estás llamando.
url_renovables = "https://apidatos.ree.es/datos/generacion/evolucion-renovable-no-renovable"

# Definir los parámetros de la consulta (si es necesario, según la documentación de la API).
params = {
    'start_date': '2021-01-01',  #Cambie las fechas gracias a los detalles del error que ponia un max de 24 meses.
    'end_date': '2022-12-31',
    'time_trunc': 'month'     
}

# Definir tus cabeceras (sin la cabecera ).
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

# Realizar la llamada a la API usando la URL base, el endpoint, las cabeceras y los parámetros
resultados_renovables = llamar_api(url_renovables, headers, params)
resultados_renovables


La llamada a la API nos ha dado una respuesta de tipo: 200


{'data': {'type': 'Evolución de la generación renovable/no renovable',
  'id': 'gen2',
  'attributes': {'title': 'Evolución de la generación renovable/no renovable',
   'last-update': '2022-01-11T09:21:30.000+01:00',
   'description': None},
  'meta': {'cache-control': {'cache': 'HIT',
    'expireAt': '2024-05-19T08:05:34'}}},
 'included': [{'type': 'Renovable',
   'id': '10299',
   'groupId': None,
   'attributes': {'title': 'Renovable',
    'description': 'Energía renovable: incluye hidráulica, hidroeólica, eólica, solar fotovoltaica, solar térmica, otras renovables y residuos renovables.',
    'color': '#92d050',
    'type': 'Renovable',
    'magnitude': None,
    'composite': False,
    'last-update': '2022-01-11T09:21:30.000+01:00',
    'values': [{'value': 12561066.857,
      'percentage': 0.5167998931450627,
      'datetime': '2021-01-01T00:00:00.000+01:00'},
     {'value': 12359937.817,
      'percentage': 0.5848258664348669,
      'datetime': '2021-02-01T00:00:00.000+01:00'},


Ahora agregamos la columna que pide el ejercicio 1:

In [54]:
 # Extraer la parte 'included' del resultado, que contiene los datos de interés
datos_incluidos =resultados_renovables['included'] #El 'included' lo saqué al ver los datos de resultados_informacion
# Crear una lista vacía para guardar los datos transformados
columna_nueva = []
# Iterar sobre cada entrada en 'included'
for entry in datos_incluidos:
    # Para cada conjunto de valores, añadir una nueva clave con el tipo de energía
    for value in entry['attributes']['values']:
        value['Tipo de Energía'] = entry['type'] # Usamos el campo 'type' para determinar si es renovable
        columna_nueva.append(value)

# Convertir la lista de datos en un DataFrame de Pandas
df = pd.DataFrame(columna_nueva)
# Mostrar el DataFrame resultante
print(df)


           value  percentage                       datetime Tipo de Energía
0   1.256107e+07    0.516800  2021-01-01T00:00:00.000+01:00       Renovable
1   1.235994e+07    0.584826  2021-02-01T00:00:00.000+01:00       Renovable
2   1.179999e+07    0.534004  2021-03-01T00:00:00.000+01:00       Renovable
3   9.240647e+06    0.464346  2021-04-01T00:00:00.000+02:00       Renovable
4   1.043336e+07    0.509653  2021-05-01T00:00:00.000+02:00       Renovable
5   9.287873e+06    0.467578  2021-06-01T00:00:00.000+02:00       Renovable
6   1.037779e+07    0.462535  2021-07-01T00:00:00.000+02:00       Renovable
7   9.182781e+06    0.425420  2021-08-01T00:00:00.000+02:00       Renovable
8   7.472118e+06    0.361166  2021-09-01T00:00:00.000+02:00       Renovable
9   7.991904e+06    0.387305  2021-10-01T00:00:00.000+02:00       Renovable
10  9.657320e+06    0.419561  2021-11-01T00:00:00.000+01:00       Renovable
11  1.109436e+07    0.464680  2021-12-01T00:00:00.000+01:00       Renovable
12  9.750045

# 2. Extracción de Datos por Comunidad Autónoma:

Extraer datos de generación de energía renovable y no renovable por Comunidad Autónoma.

Utilizar el diccionario cod_comunidades para obtener los códigos de cada comunidad autónoma.

Transformar los datos para incluir una columna que especifique la comunidad autónoma y su identificador.


El diccionario de cod_comunidades es:


/es/datos/generacion/evolucion-renovable-no-renovable/geo_limit=peninsular&geo_ids=8744,8745,4,5,6,7,8,9,10,11,13,14,15,16,17,8743,8742,21,20

In [55]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

# ME DA FALLO DEL SERVIDOR
La llamada a la API nos ha dado una respuesta de tipo: 500
El motivo por el que la llamada falló es Internal Server Error
Detalles del error: {"errors":[{"code":5,"status":"500","title":"Internal server error","detail":"No se pudo conectar al servidor de datos."}]

In [56]:
def llamar_api(url, headers, params = None):
    # Realizar la solicitud GET pasando la URL y los encabezados.
    llamada = requests.get(url, headers=headers,params=params)
    # Imprimir el código de estado de la respuesta para verificar si fue exitosa.
    print(f"La llamada a la API nos ha dado una respuesta de tipo: {llamada.status_code}")
    # Si el código de estado es diferente de 200, imprimir la razón del fallo.
    if llamada.status_code != 200:
        print(f"El motivo por el que la llamada falló es {llamada.reason}")
        print("Detalles del error:", llamada.text)   #llamada.text me da detalles más especificos del error y así poder arreglarlo.
    else:
        # Si la llamada fue exitosa, devolver la respuesta en formato JSON.
        return llamada.json()  

#Definimos url Base.
url_red_electrica= "https://apidatos.ree.es"   

# Definir el endpoint específico que estás llamando.
url_renovables = "/es/datos/generacion/evolucion-renovable-no-renovable"

params = {
'geo_limit': 'peninsular,canarias,baleares,ceuta,melilla,ccaa ',
'geo_ids': '8744,8745,4,5,6,7,8,9,10,11,13,14,15,16,17,8743,8742,21,20',
'start_date': '2021-01-01',  # Asegúrate de incluir las fechas de inicio y fin
    'end_date': '2021-12-31',
    'time_trunc': 'month'
}

# Definir tus cabeceras (sin la cabecera de Host).
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

resultados_renovables = llamar_api(url_red_electrica+ url_renovables, headers, params)
resultados_renovables

La llamada a la API nos ha dado una respuesta de tipo: 500
El motivo por el que la llamada falló es Internal Server Error
Detalles del error: {"errors":[{"code":5,"status":"500","title":"Internal server error","detail":"No se pudo conectar al servidor de datos."}]}
